In [2]:
import pandas as pd
import numpy as np
from tensorflow.keras import Sequential, layers, models
from sklearn.preprocessing import MinMaxScaler, RobustScaler
from tensorflow.keras.callbacks import EarlyStopping
import h5py as h5
import matplotlib.pyplot as plt

2023-06-12 23:21:05.983386: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-12 23:21:06.261890: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-06-12 23:21:06.962388: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-06-12 23:21:06.962477: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or 

In [3]:
y_df = pd.read_csv("../data/y_dataframe.csv")
images_file = h5.File("../data/nsd_stimuli.hdf5", "r")
images = images_file['imgBrick']
stim_list_test=np.load('../data/nsd_test_stim_list1.npy')
stim_list_train=np.load('../data/subj01_nsd_train_stim_list1.npy')
train_caps=np.load('../data/nsd_train_cap_sub1.npy')
test_caps=np.load('../data/nsd_test_cap_sub1.npy')
X_test_dataframe = pd.read_csv('../data/df_test.csv')

/tmp/ipykernel_663/1355504315.py:1: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  y_df = pd.read_csv("../data/y_dataframe.csv")


In [4]:
y_train = y_df.iloc[:8859, 1:].to_numpy()

In [5]:
y_df

Unnamed: 0  1950s  1970  1971  2009  4-way  abandon  about  above  \
0                 0      0     0     0     0      0        0      0      0   
1                 1      0     0     0     0      0        0      0      0   
2                 2      0     0     0     0      0        0      0      0   
3                 3      0     0     0     0      0        0      0      0   
4                 4      0     0     0     0      0        0      0      0   
...             ...    ...   ...   ...   ...    ...      ...    ...    ...   
9836  X_test_id_977      0     0     0     0      0        0      0      0   
9837  X_test_id_978      0     0     0     0      0        0      0      0   
9838  X_test_id_979      0     0     0     0      0        0      0      0   
9839  X_test_id_980      0     0     0     0      0        0      0      0   
9840  X_test_id_981      0     0     0     0      0        0      0      0   

      abraham  ...  yogurt  york  you  young  younger  youth  zebra  zip  \
0           0  ...       0     0    0      0        0      0      0    0   
1           0  ...       0     0    0      0        0      0      0    0   
2           0  ...       0     0    0      0        0      0      0    0   
3           0  ...       0     0    0      0        0      0      0    0   
4           0  ...       0     0    0      0        0      0      1    0   
...       ...  ...     ...   ...  ...    ...      ...    ...    ...  ...   
9836        0  ...       0     0    0      0        0      0      0    0   
9837        0  ...       0     0    0      0        0      0      0    0   
9838        0  ...       0     0    0      0        0      0      0    0   
9839        0  ...       0     0    0      0        0      0      0    0   
9840        0  ...       0     0    0      0        0      0      0    0   

      zombie  zoo  
0          0    0  
1          0    0  
2          0    0  
3          0    0  
4          0    0  
...      ...  ...  
9836       0    0  
9837       0    0  
9838       0    0  
9839       0    0  
9840       0    0  

[9841 rows x 2638 columns]

In [6]:
y_df.drop(columns = ["Unnamed: 0"], inplace = True)

In [7]:
X_train = np.load("../data/nsd_train_fmriavg_nsdgeneral_sub1.npy")

In [8]:
# from sklearn.preprocessing import RobustScaler

In [9]:
scaler = RobustScaler()

In [10]:
X_scaled = scaler.fit_transform(X_train)

In [11]:
# from tensorflow.keras import Sequential
# from tensorflow.keras import layers

In [12]:
model = Sequential([layers.Dense(1000, input_shape = (15724,), activation = "relu"),
                  layers.Dense(1000, activation = "relu"),
                    layers.Dropout(0.5),
                  layers.Dense(2000, activation = "relu"),
                    layers.Dropout(0.5),
                  layers.Dense(3000, activation = "relu"),
                    layers.Dropout(0.5),
                    layers.Dense(5000, activation = "relu"),
                    layers.Dropout(0.5),
                    layers.Dense(1000, activation = "relu"),
                    layers.Dropout(0.5),
                    layers.Dense(2000, activation = "relu"),
                  layers.Dense(2637, activation = "sigmoid")])

2023-06-12 23:21:27.525876: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:966] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-06-12 23:21:27.653473: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudnn.so.8'; dlerror: libcudnn.so.8: cannot open shared object file: No such file or directory
2023-06-12 23:21:27.653487: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1934] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
2023-06-12 23:21:27.654953: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (o

In [13]:
model.compile(loss = "binary_crossentropy", optimizer = "adam")

In [14]:
# es = EarlyStopping(patience=7,restore_best_weights = True)

# history = model.fit(X_scaled, y_train, validation_split=0.1, batch_size = 32, epochs = 40, callbacks=[es])

In [ ]:
history = model.fit(X_scaled, y_train, validation_split=0.1, batch_size = 32, epochs = 30)

In [ ]:
X_test = np.load("../data/nsd_test_fmriavg_nsdgeneral_sub1.npy")

In [ ]:
X_test_scaled = scaler.transform(X_test)

In [ ]:
idx=15
test_answer = model.predict(X_test_scaled[idx].reshape(1,-1))

In [ ]:
labels = y_df.columns

In [ ]:
labels

In [ ]:
test_answer

In [ ]:
test_answer[test_answer>0.4]

In [ ]:
test_answer_idxs = np.argpartition(test_answer[0],-4)[-4:]

In [ ]:
labels[test_answer_idxs]

In [ ]:
# idx=300
# test_answer = model.predict(X_test_scaled[idx].reshape(1,-1))
# labels = y_df.columns

# test_answer_idxs = np.argpartition(test_answer[0],-2)[-2:]
# labels[test_answer_idxs]

In [ ]:
df_test_idx_caps=X_test_dataframe[['Image_index', 'test_caps_processed']]



In [ ]:
# idx=333
# test_index=stim_list_test[idx]
# test_index

In [ ]:
# idx=51
# test_answer = model.predict(X_test_scaled[idx].reshape(1,-1))
# test_answer_idxs = np.argpartition(test_answer[0],-4)[-4:]
# labels[test_answer_idxs]

In [ ]:
pwd

In [ ]:
# import pickle

# # Load the model from a file
# with open('model.pkl', 'rb') as file:
# model = pickle.load(file)


In [ ]:
idx=1
test_answer = model.predict(X_test_scaled[idx].reshape(1,-1))
test_answer_idxs = np.argpartition(test_answer[0],-4)[-4:]
labels[test_answer_idxs]
test_index=stim_list_test[idx]
caps=df_test_idx_caps.loc[df_test_idx_caps['Image_index'] == test_index, 'test_caps_processed'].item()
plt.imshow(images[test_index])
plt.show()
print('Original captions', test_caps[idx])
print('Test captions processed:' , caps)
# print('fMRI array', X_test[idx])
print('Predicted labels', list(labels[test_answer_idxs]))


In [ ]:
X_test_dataframe

In [ ]:
## Save the model to a file
# with open('model.pkl', 'wb') as file:
#     pickle.dump(model, file)

In [ ]:
pwd

In [ ]:
models.save_model(model, '../model/models/my_model_fat')